In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

from IPython.display import display, HTML

#CSS = """
#.output {
#    flex-direction: row;
#}
#"""

#HTML('<style>{}</style>'.format(CSS))

In [2]:
number_of_iteration=10
cwd=os.getcwd()
visual_m_path = os.path.join(cwd,'data/imgs_mean_feature_vectors.pkl')
textual_m_path = os.path.join(cwd,'data/docs_extracted_features.pkl')
models_folder_name = os.path.join(cwd,'models',str(number_of_iteration))
path_to_save_test_results=os.path.join(models_folder_name, 'test_results.pkl')
model_checkpoint_path = os.path.join(cwd,models_folder_name,'gmu.ckpt')
models_folder_name_test = os.path.join(cwd,os.pardir,'text_feature_extraction','models','test',str(number_of_iteration))
path_to_preprocessed_texts_test=os.path.join(models_folder_name_test,'recipes_test_dataset.pkl')


df_visual_m = pd.read_pickle(visual_m_path)
df_textual_m = pd.read_pickle(textual_m_path)

number_of_recipes=len(df_visual_m)
unique_labels=sorted(set(df_visual_m.mean_vector_labels.values))
number_of_classes=len(unique_labels)
possible_class_indices=list(range(0,number_of_classes))
labels2class_indices=dict(zip(unique_labels,possible_class_indices))
print(labels2class_indices)

#np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

{'chocolate_mousse': 5, 'tacos': 24, 'garlic_bread': 11, 'frozen_yogurt': 10, 'sushi': 23, 'chicken_wings': 3, 'tiramisu': 25, 'deviled_eggs': 6, 'hamburger': 12, 'waffles': 26, 'french_toast': 8, 'cheesecake': 1, 'pizza': 20, 'fried_rice': 9, 'macarons': 16, 'nachos': 17, 'macaroni_and_cheese': 15, 'chocolate_cake': 4, 'steak': 22, 'hummus': 13, 'chicken_curry': 2, 'pancakes': 19, 'apple_pie': 0, 'lasagna': 14, 'onion_rings': 18, 'dumplings': 7, 'sashimi': 21}


In [3]:
not_normalized_features_visual=df_visual_m.mean_feature_vectors.tolist()
mean_visual=np.mean(not_normalized_features_visual)
print("Mean value: visual features ", mean_visual)
subtracted_mean_visual= (not_normalized_features_visual-mean_visual)
max_visual=np.max(subtracted_mean_visual)
print("Max value in visual feature vectors after subraction of mean value: ", max_visual)
normalized_features_visual = (subtracted_mean_visual/max_visual).tolist()
df_visual_m.mean_feature_vectors=normalized_features_visual

not_normalized_features_textual=df_textual_m.doc_embeddings.tolist()
mean_textual=np.mean(not_normalized_features_textual)
print("Mean value: textual features ", mean_textual)
subtracted_mean_textual= (not_normalized_features_textual-mean_textual)
max_textual=np.max(subtracted_mean_textual)
print("Max value in textual feature vectors after subraction of mean value: ", max_textual)
normalized_features_textual = (subtracted_mean_textual/max_textual).tolist()
df_textual_m.doc_embeddings=normalized_features_textual


df_textual_m


Mean value: visual features  0.5558735478366387
Max value in visual feature vectors after subraction of mean value:  22.56199106092801
Mean value: textual features  0.013499930763029529
Max value in textual feature vectors after subraction of mean value:  3.519134565925813


,doc_embeddings,labels,text_names
0,"[-0.11329869829923188, -0.6441225998849662, -0...",apple_pie,1.txt
1,"[-0.07474968697600654, -0.28306451085312634, 0...",apple_pie,10.txt
2,"[-0.10599670561622081, -0.5485997736729018, 0....",apple_pie,11.txt
3,"[-0.5233624146389693, -0.32503846481726917, 0....",apple_pie,12.txt
4,"[-0.07889076414967888, -0.2116037304174958, 0....",apple_pie,13.txt
5,"[-0.07946860561194324, -0.2566648633469014, -0...",apple_pie,14.txt
6,"[-0.1800028915393702, -0.6000170525947526, 0.1...",apple_pie,15.txt
7,"[-0.28211158437034184, -0.503643364081505, 0.2...",apple_pie,2.txt
8,"[-0.41500734646954496, -0.40415873356834714, 0...",apple_pie,3.txt
9,"[-0.38648066177147605, -0.48593944596184546, 0...",apple_pie,4.txt


In [4]:
df_preprocessed_texts_test = pd.read_pickle(path_to_preprocessed_texts_test)
test_samples=df_preprocessed_texts_test.index.values.tolist()

#test_samples=[3,5,6,
#              11,17,18,
#              23,26,33,34,
#              38,39,44
#             ]
all_samples=set(range(0,number_of_recipes))
train_samples=list(all_samples.difference(test_samples))
print("train samples: ", train_samples)
print("test samples: ", test_samples)

train samples:  [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 148, 149, 150, 151, 153, 154, 155, 156, 158, 159, 160, 161, 162, 164, 165, 167, 168, 169, 170, 171, 173, 174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 198, 199, 200, 201, 203, 204, 205, 206, 207, 208, 209, 211, 212, 213, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 234, 235, 236, 237, 238, 240, 241, 243, 245, 247, 248, 249, 250, 251, 252, 253, 254, 255,

In [5]:
df_visual_m_train=df_visual_m.iloc[train_samples]
df_visual_m_test=df_visual_m.iloc[test_samples]
df_textual_m_train=df_textual_m.iloc[train_samples]
df_textual_m_test=df_textual_m.iloc[test_samples]

visual_m_train_inputs=list(df_visual_m_train.mean_feature_vectors.values)
visual_m_test_inputs=list(df_visual_m_test.mean_feature_vectors.values)

textual_m_train_inputs=list(df_textual_m_train.doc_embeddings.values)
textual_m_test_inputs=list(df_textual_m_test.doc_embeddings.values)

train_correct_class_ids=[labels2class_indices[l] for l in df_visual_m_train.mean_vector_labels]
test_correct_class_ids=[labels2class_indices[l] for l in df_visual_m_test.mean_vector_labels]

number_of_training_samples=len(visual_m_train_inputs)
number_of_test_samples=len(visual_m_test_inputs)
len_of_visual_features_vec=len(visual_m_train_inputs[0])
len_of_textual_features_vec=len(textual_m_train_inputs[0])

print(test_correct_class_ids)
print(np.shape(visual_m_train_inputs))
print(np.shape(textual_m_train_inputs))

[0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 4, 5, 6, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 13, 13, 14, 14, 15, 15, 15, 16, 17, 17, 18, 19, 20, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 25, 25, 26, 26]
(264, 202)
(264, 27)


In [6]:
batch_size=4
learning_rate=0.001
hidden_state_dim = 27
#z_dim=1
number_of_training_iterations=20000
print_valid_every=20
num_repeat_training=10

In [7]:
df_textual_m_test

,doc_embeddings,labels,text_names
0,"[-0.11329869829923188, -0.6441225998849662, -0...",apple_pie,1.txt
4,"[-0.07889076414967888, -0.2116037304174958, 0....",apple_pie,13.txt
8,"[-0.41500734646954496, -0.40415873356834714, 0...",apple_pie,3.txt
20,"[0.0018196716644160666, -0.09403709886171152, ...",cheesecake,4.txt
22,"[-0.37221488045095963, 0.15941117175552535, -0...",cheesecake,6.txt
28,"[-0.47327678027169323, -0.011298772057821004, ...",chicken_curry,3.txt
38,"[0.06220996901184445, 0.3393704623708661, 0.49...",chicken_wings,13.txt
39,"[0.23359608008893995, 0.1726547022163415, 0.32...",chicken_wings,14.txt
47,"[0.2844618296471188, 0.3743878599254398, 0.252...",chicken_wings,6.txt
54,"[0.049437621588535006, 0.3873440151839275, 0.3...",chocolate_cake,3.txt


In [8]:
def create_training_batch():
    inputs_visual=[]
    inputs_textual=[]
    correct_classes=[]
    for i in range(batch_size):
        train_sample_index=np.random.choice(range(0,number_of_training_samples),1)[0]
        inputs_visual.append(visual_m_train_inputs[train_sample_index])
        inputs_textual.append(textual_m_train_inputs[train_sample_index])
        correct_classes.append(train_correct_class_ids[train_sample_index])
    return np.array(inputs_visual),np.array(inputs_textual),np.array(correct_classes)

inputs_visual,inputs_textual,correct_classes=create_training_batch()
print(np.array(correct_classes))
print(np.shape(inputs_visual))
print(np.shape(inputs_textual))
print(np.shape(correct_classes))

[17 11  4 18]
(4, 202)
(4, 27)
(4,)


In [9]:
visual = tf.placeholder(tf.float32, shape=[None,len_of_visual_features_vec])
textual = tf.placeholder(tf.float32, shape=[None,len_of_textual_features_vec])
target = tf.placeholder(tf.int32, shape=[None])

#v_reduced = tf.layers.dense(visual,
#                      hidden_state_dim,
##                      activation=tf.nn.tanh)
#                       activation=tf.nn.relu)


h_v = tf.layers.dense(visual,
                      hidden_state_dim,
                      activation=tf.nn.tanh)


h_t = tf.layers.dense(textual,
                      hidden_state_dim,
                      activation=tf.nn.tanh)

z = tf.layers.dense(tf.concat([visual,textual], axis=1),
                    hidden_state_dim,
                    kernel_initializer=tf.initializers.random_uniform,
                    activation=tf.nn.sigmoid)

#h = tf.concat([z * h_v,(1 - z) * h_t], axis=1)
h = tf.concat([(1-z) * h_v, z * h_t], axis=1)
#h = z * h_v + (1 - z) * h_t
#h = (1-z) * h_v +  z * h_t


logits = tf.layers.dense(h, number_of_classes)
scores = tf.nn.sigmoid(logits)

multi_class_labels=tf.one_hot(target, depth=number_of_classes)
loss = tf.reduce_mean(tf.losses.sigmoid_cross_entropy(multi_class_labels=multi_class_labels,
                                       logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

correct_prediction=tf.equal(tf.argmax(logits, axis=1), tf.argmax(multi_class_labels,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


saver = tf.train.Saver()

In [10]:
best_accuracy=0

def show_validation_result(session,accuracy_res, loss_res, scores_res,z_res):
    print("Validation: Loss: ", loss_res," Accuracy: ", accuracy_res)
    global best_accuracy
    if accuracy_res > best_accuracy:
        best_accuracy=accuracy_res
        df_performance=pd.DataFrame(data={'Accuracy':[accuracy_res], 'Loss':[loss_res]})
        class_scores=list(np.around(scores_res,4))
        predicted_labels = [np.argmax(one_recipe_class_scores) for one_recipe_class_scores in class_scores]
        df_scores=pd.DataFrame(data={'Class scores':class_scores,
                               'Predicted labels':predicted_labels,
                               'Correct labels':test_correct_class_ids,
                               'Trust to visual modality': list(np.around(1-z_res,4)),
                               'Trust to textual modality': list(np.around(z_res,4))})
        df_res=pd.concat([df_scores, df_performance], axis=1)
        df_res.to_pickle(path_to_save_test_results)
        saver.save(session, model_checkpoint_path)
        display(df_res)
        #print(df_scores)
        
    

def train(num_of_run):
    #with tf.Session() as session:
        session=tf.InteractiveSession()
        session.run(tf.global_variables_initializer())
    
        print("Start model training")
    
        for train_iter in range(number_of_training_iterations):
            if(train_iter==0):
                accuracy_res, loss_res, scores_res,z_res = session.run([accuracy, loss, scores, z],
                                                                          {visual: visual_m_test_inputs,
                                                                           textual: textual_m_test_inputs,
                                                                           target: test_correct_class_ids})        
                
                show_validation_result(session,accuracy_res, loss_res, scores_res,z_res)
            
            inputs_visual,inputs_textual,correct_classes=create_training_batch()
            _, l = session.run([train_op, loss], {visual: inputs_visual,
                                           textual: inputs_textual,
                                           target: correct_classes})
            print(train_iter, ": Training: loss: ", l)
        
            if (train_iter)%print_valid_every==0:
                accuracy_res, loss_res, scores_res,z_res = session.run([accuracy, loss, scores, z],
                                                                          {visual: visual_m_test_inputs,
                                                                           textual: textual_m_test_inputs,
                                                                           target: test_correct_class_ids})        
                
                show_validation_result(session,accuracy_res, loss_res, scores_res,z_res)
                
                if accuracy_res==1.0:
                    #saver.save(session, model_checkpoint_path)
                    return 0
                
        #saver.save(session, model_checkpoint_path)
        session.close()
        return -1

In [11]:
for num_of_run in range(num_repeat_training):            
    if train(num_of_run)==0:
        break
        

Start model training
Validation: Loss:  0.6955  Accuracy:  0.01923077


,Class scores,Correct labels,Predicted labels,Trust to textual modality,Trust to visual modality,Accuracy,Loss
0,"[0.4993, 0.5091, 0.5001, 0.4981, 0.5227, 0.494...",0,4,"[0.2245, 0.324, 0.1933, 0.2098, 0.1259, 0.1964...","[0.7755, 0.676, 0.8067, 0.7902, 0.8741, 0.8036...",0.019231,0.6955
1,"[0.5201, 0.5361, 0.5074, 0.4937, 0.4886, 0.451...",0,23,"[0.6778, 0.6248, 0.4293, 0.5012, 0.3995, 0.396...","[0.3222, 0.3752, 0.5707, 0.4988, 0.6005, 0.603...",NaN,NaN
2,"[0.4869, 0.5057, 0.5097, 0.4806, 0.5153, 0.505...",0,19,"[0.4112, 0.3977, 0.2679, 0.2338, 0.146, 0.2432...","[0.5888, 0.6023, 0.7321, 0.7662, 0.854, 0.7568...",NaN,NaN
3,"[0.4891, 0.4835, 0.5483, 0.5001, 0.4607, 0.477...",1,19,"[0.5915, 0.4715, 0.4486, 0.5585, 0.6776, 0.546...","[0.4085, 0.5285, 0.5514, 0.4415, 0.3224, 0.453...",NaN,NaN
4,"[0.5094, 0.5011, 0.4949, 0.4948, 0.5098, 0.499...",1,24,"[0.157, 0.1306, 0.144, 0.1306, 0.1126, 0.0795,...","[0.843, 0.8694, 0.856, 0.8694, 0.8874, 0.9205,...",NaN,NaN
5,"[0.5026, 0.5062, 0.514, 0.4961, 0.5092, 0.5213...",2,24,"[0.1125, 0.095, 0.1106, 0.1575, 0.1214, 0.2062...","[0.8875, 0.905, 0.8894, 0.8425, 0.8786, 0.7938...",NaN,NaN
6,"[0.5407, 0.5242, 0.4296, 0.4623, 0.5959, 0.475...",3,4,"[0.8742, 0.6417, 0.8277, 0.7006, 0.6697, 0.688...","[0.1258, 0.3583, 0.1723, 0.2994, 0.3303, 0.312...",NaN,NaN
7,"[0.4794, 0.4752, 0.4821, 0.4682, 0.5023, 0.528...",3,5,"[0.7048, 0.4036, 0.6063, 0.4072, 0.4417, 0.581...","[0.2952, 0.5964, 0.3937, 0.5928, 0.5583, 0.418...",NaN,NaN
8,"[0.4818, 0.5047, 0.4855, 0.4723, 0.53, 0.5514,...",3,5,"[0.4765, 0.2927, 0.4979, 0.3415, 0.3083, 0.472...","[0.5235, 0.7073, 0.5021, 0.6585, 0.6917, 0.527...",NaN,NaN
9,"[0.4994, 0.4958, 0.5786, 0.4333, 0.4923, 0.427...",4,2,"[0.9025, 0.9208, 0.9457, 0.8855, 0.9428, 0.881...","[0.0975, 0.0792, 0.0543, 0.1145, 0.0572, 0.118...",NaN,NaN


0 : Training: loss:  0.6979154
Validation: Loss:  0.6943165  Accuracy:  0.01923077
1 : Training: loss:  0.69063485
2 : Training: loss:  0.6926439
3 : Training: loss:  0.7002935
4 : Training: loss:  0.6948166
5 : Training: loss:  0.68777055
6 : Training: loss:  0.6936111
7 : Training: loss:  0.6803202
8 : Training: loss:  0.6789374
9 : Training: loss:  0.68382037
10 : Training: loss:  0.6766239
11 : Training: loss:  0.6792958
12 : Training: loss:  0.6808055
13 : Training: loss:  0.6763594
14 : Training: loss:  0.6795404
15 : Training: loss:  0.6769553
16 : Training: loss:  0.6707012
17 : Training: loss:  0.67910796
18 : Training: loss:  0.68152934
19 : Training: loss:  0.65986633
20 : Training: loss:  0.6739189
Validation: Loss:  0.6698643  Accuracy:  0.01923077
21 : Training: loss:  0.6649807
22 : Training: loss:  0.6723937
23 : Training: loss:  0.6514553
24 : Training: loss:  0.67350245
25 : Training: loss:  0.65896404
26 : Training: loss:  0.6518119
27 : Training: loss:  0.65791136
2

,Class scores,Correct labels,Predicted labels,Trust to textual modality,Trust to visual modality,Accuracy,Loss
0,"[0.4583, 0.4908, 0.434, 0.4648, 0.4541, 0.4481...",0,1,"[0.2089, 0.3206, 0.1865, 0.2111, 0.1282, 0.192...","[0.7911, 0.6794, 0.8135, 0.7889, 0.8718, 0.807...",0.038462,0.631157
1,"[0.4879, 0.5043, 0.4379, 0.4536, 0.4306, 0.406...",0,23,"[0.6566, 0.6141, 0.423, 0.5091, 0.3898, 0.3907...","[0.3434, 0.3859, 0.577, 0.4909, 0.6102, 0.6093...",NaN,NaN
2,"[0.4489, 0.4869, 0.4447, 0.4476, 0.4503, 0.457...",0,19,"[0.3883, 0.3963, 0.2634, 0.2348, 0.1473, 0.236...","[0.6117, 0.6037, 0.7366, 0.7652, 0.8527, 0.763...",NaN,NaN
3,"[0.4514, 0.4547, 0.4806, 0.4637, 0.4063, 0.427...",1,19,"[0.563, 0.4692, 0.4344, 0.5549, 0.6724, 0.5445...","[0.437, 0.5308, 0.5656, 0.4451, 0.3276, 0.4555...",NaN,NaN
4,"[0.4573, 0.48, 0.4191, 0.4581, 0.4339, 0.4475,...",1,8,"[0.1421, 0.1277, 0.1393, 0.1275, 0.1148, 0.078...","[0.8579, 0.8723, 0.8607, 0.8725, 0.8852, 0.921...",NaN,NaN
5,"[0.4568, 0.4908, 0.4548, 0.4654, 0.445, 0.4779...",2,1,"[0.1092, 0.0973, 0.1072, 0.1525, 0.1209, 0.202...","[0.8908, 0.9027, 0.8928, 0.8475, 0.8791, 0.797...",NaN,NaN
6,"[0.5157, 0.4899, 0.3742, 0.445, 0.5275, 0.4159...",3,4,"[0.8627, 0.6587, 0.8264, 0.7138, 0.6568, 0.692...","[0.1373, 0.3413, 0.1736, 0.2862, 0.3432, 0.307...",NaN,NaN
7,"[0.4481, 0.452, 0.432, 0.4475, 0.4472, 0.4824,...",3,20,"[0.6906, 0.4158, 0.6059, 0.4131, 0.4309, 0.582...","[0.3094, 0.5842, 0.3941, 0.5869, 0.5691, 0.417...",NaN,NaN
8,"[0.4537, 0.4894, 0.4422, 0.4563, 0.4781, 0.511...",3,5,"[0.4668, 0.3037, 0.5, 0.3478, 0.3003, 0.4651, ...","[0.5332, 0.6963, 0.5, 0.6522, 0.6997, 0.5349, ...",NaN,NaN
9,"[0.48, 0.4691, 0.5416, 0.4138, 0.4631, 0.4003,...",4,26,"[0.8936, 0.9199, 0.942, 0.8838, 0.9415, 0.8746...","[0.1064, 0.0801, 0.0579, 0.1162, 0.0585, 0.125...",NaN,NaN


41 : Training: loss:  0.61920094
42 : Training: loss:  0.62100536
43 : Training: loss:  0.61620194
44 : Training: loss:  0.60918313
45 : Training: loss:  0.6052737
46 : Training: loss:  0.6182846
47 : Training: loss:  0.59938204
48 : Training: loss:  0.6194034
49 : Training: loss:  0.58511853
50 : Training: loss:  0.60338944
51 : Training: loss:  0.59903115
52 : Training: loss:  0.5914879
53 : Training: loss:  0.59200746
54 : Training: loss:  0.60962605
55 : Training: loss:  0.5960706
56 : Training: loss:  0.5965242
57 : Training: loss:  0.5867036
58 : Training: loss:  0.5907285
59 : Training: loss:  0.56980044
60 : Training: loss:  0.57791597
Validation: Loss:  0.5729713  Accuracy:  0.057692308


...

11223 : Training: loss:  0.00032902186
11224 : Training: loss:  0.00027929596
11225 : Training: loss:  0.00021164316
11226 : Training: loss:  0.00051468797
11227 : Training: loss:  0.00040628997
11228 : Training: loss:  0.0003328049
11229 : Training: loss:  0.00030960015
11230 : Training: loss:  0.00028274854
11231 : Training: loss:  0.0003010003
11232 : Training: loss:  0.0002998447
11233 : Training: loss:  0.00020076966
11234 : Training: loss:  0.00026994
11235 : Training: loss:  0.00040745878
11236 : Training: loss:  0.00024402847
11237 : Training: loss:  0.0003825044
11238 : Training: loss:  0.00014415111
11239 : Training: loss:  0.00019819933
11240 : Training: loss:  0.00019037808
Validation: Loss:  0.012598564  Accuracy:  0.96153843
11241 : Training: loss:  0.00038864336
11242 : Training: loss:  0.0002485583
11243 : Training: loss:  0.00023606785
11244 : Training: loss:  0.00027262757
11245 : Training: loss:  0.00026738725
11246 : Training: loss:  0.000149326
11247 : Training: lo

11421 : Training: loss:  0.00021613787
11422 : Training: loss:  0.00016635982
11423 : Training: loss:  0.0002762349
11424 : Training: loss:  0.00016277489
11425 : Training: loss:  0.00022049571
11426 : Training: loss:  0.00021856575
11427 : Training: loss:  0.00020497358
11428 : Training: loss:  0.00010329233
11429 : Training: loss:  0.00038736954
11430 : Training: loss:  0.0002203295
11431 : Training: loss:  0.00021848579
11432 : Training: loss:  0.00010747173
11433 : Training: loss:  0.00025511015
11434 : Training: loss:  0.000380809
11435 : Training: loss:  0.00019075669
11436 : Training: loss:  0.00022559868
11437 : Training: loss:  0.00018862206
11438 : Training: loss:  0.0005007968
11439 : Training: loss:  0.0002668855
11440 : Training: loss:  0.00039332014
Validation: Loss:  0.012760654  Accuracy:  0.96153843
11441 : Training: loss:  0.0003570446
11442 : Training: loss:  0.00033576452
11443 : Training: loss:  0.0002884805
11444 : Training: loss:  0.00021415623
11445 : Training: 

11623 : Training: loss:  0.00016497329
11624 : Training: loss:  0.00025081256
11625 : Training: loss:  0.00010293853
11626 : Training: loss:  0.00025978938
11627 : Training: loss:  0.00019783882
11628 : Training: loss:  0.00012897185
11629 : Training: loss:  0.00020697011
11630 : Training: loss:  0.00025504155
11631 : Training: loss:  0.00016741801
11632 : Training: loss:  0.00020410342
11633 : Training: loss:  0.00027210542
11634 : Training: loss:  0.0002734748
11635 : Training: loss:  0.00041277608
11636 : Training: loss:  0.00014146857
11637 : Training: loss:  0.00022043737
11638 : Training: loss:  0.00018259602
11639 : Training: loss:  0.00013710509
11640 : Training: loss:  0.0001892343
Validation: Loss:  0.012286985  Accuracy:  0.96153843
11641 : Training: loss:  0.00016031926
11642 : Training: loss:  7.991305e-05
11643 : Training: loss:  0.00021554582
11644 : Training: loss:  5.1606396e-05
11645 : Training: loss:  0.00015252113
11646 : Training: loss:  0.00029995912
11647 : Train

11836 : Training: loss:  0.00024669667
11837 : Training: loss:  0.00018503002
11838 : Training: loss:  0.00016223191
11839 : Training: loss:  0.00019277858
11840 : Training: loss:  0.0002230406
Validation: Loss:  0.01236864  Accuracy:  0.9807692
11841 : Training: loss:  0.00015234562
11842 : Training: loss:  0.0002568887
11843 : Training: loss:  0.00016471281
11844 : Training: loss:  0.00017161509
11845 : Training: loss:  0.00016363034
11846 : Training: loss:  0.00023720953
11847 : Training: loss:  0.0002263257
11848 : Training: loss:  0.0001813358
11849 : Training: loss:  0.00017857517
11850 : Training: loss:  0.00017523173
11851 : Training: loss:  5.5407425e-05
11852 : Training: loss:  0.00022120337
11853 : Training: loss:  0.00011488406
11854 : Training: loss:  0.00014841926
11855 : Training: loss:  0.00019140905
11856 : Training: loss:  0.0002759958
11857 : Training: loss:  0.00017910378
11858 : Training: loss:  0.00017057781
11859 : Training: loss:  0.000102542916
11860 : Training

,Class scores,Correct labels,Predicted labels,Trust to textual modality,Trust to visual modality,Accuracy,Loss
0,"[0.3605, 0.0056, 0.0, 0.0, 0.0, 1e-04, 0.0, 0....",0,0,"[0.5667, 0.8508, 0.0415, 0.8179, 0.5946, 0.032...","[0.4333, 0.1492, 0.9585, 0.1821, 0.4054, 0.967...",1.0,0.012327
1,"[0.9978, 0.1794, 0.0, 0.0, 0.002, 0.0, 0.0, 0....",0,0,"[0.916, 0.9429, 0.1123, 0.7217, 0.9216, 0.2132...","[0.084, 0.0571, 0.8877, 0.2783, 0.0784, 0.7868...",NaN,NaN
2,"[0.9986, 0.0037, 0.0, 0.0, 0.0, 0.0006, 0.0, 0...",0,0,"[0.6383, 0.961, 0.1062, 0.9306, 0.6189, 0.0528...","[0.3617, 0.039, 0.8938, 0.0694, 0.3811, 0.9472...",NaN,NaN
3,"[0.0, 0.8612, 0.0, 0.0, 0.0023, 0.0002, 0.0, 0...",1,1,"[0.5619, 0.6391, 0.0503, 0.8094, 0.9924, 0.595...","[0.4381, 0.3609, 0.9497, 0.1906, 0.0076, 0.404...",NaN,NaN
4,"[1e-04, 0.8416, 0.0, 0.0, 0.0, 0.0, 1e-04, 0.0...",1,1,"[0.1703, 0.6129, 0.0142, 0.4832, 0.8252, 0.055...","[0.8297, 0.3871, 0.9858, 0.5168, 0.1748, 0.944...",NaN,NaN
5,"[0.0, 1e-04, 0.9962, 0.0, 0.0, 0.0, 1e-04, 0.0...",2,2,"[0.4713, 0.1222, 0.0621, 0.3696, 0.4155, 0.979...","[0.5287, 0.8778, 0.9379, 0.6304, 0.5845, 0.020...",NaN,NaN
6,"[0.0002, 0.0, 0.0, 0.9956, 0.0, 0.0, 0.0, 0.0,...",3,3,"[0.9934, 0.9964, 0.9655, 0.9968, 0.0585, 0.953...","[0.0066, 0.0036, 0.0345, 0.0032, 0.9415, 0.046...",NaN,NaN
7,"[0.0, 0.0, 0.0, 0.9716, 0.0, 0.0, 0.0, 0.0, 0....",3,3,"[0.9412, 0.5835, 0.8056, 0.7867, 0.174, 0.9731...","[0.0588, 0.4165, 0.1944, 0.2133, 0.826, 0.0269...",NaN,NaN
8,"[0.0002, 0.0, 0.0, 0.9945, 1e-04, 0.0, 0.0, 0....",3,3,"[0.9676, 0.9739, 0.9827, 0.5247, 0.2469, 0.939...","[0.0324, 0.0261, 0.0173, 0.4753, 0.7531, 0.060...",NaN,NaN
9,"[0.0, 0.0, 0.0, 0.0004, 0.9979, 0.0023, 0.0, 0...",4,4,"[0.9717, 0.9764, 0.8071, 0.4996, 0.9984, 0.846...","[0.0283, 0.0236, 0.1929, 0.5004, 0.0016, 0.154...",NaN,NaN
